# Imports

In [ ]:
# !pip install yfinance
# !pip install pmdarima
# !pip install hyperopt
# !pip install xgboost

In [1]:
import os
from functools import partial
from datetime import datetime

import typing
from typing import Dict
from typing import Any
from typing import Tuple

import numpy as np
import pandas as pd
import sklearn

import utils
import pipelines
import processing
import evaluate
import predict
import preprocessing

# Download stock daily prices & indexes

In [14]:
### Gets all ticker names (no argument given)
ticker_list = utils.get_ticker_names(market_cap_min_mm=1000, market_cap_max_mm=None)

In [15]:
### Specific date - 3rd of March 2022 (Y, M, D)
# date_to = datetime(2021, 1, 18)
### Date of today
date_to = datetime.today()
### How many years' of data to download (going backwards from date_end). Year can be a floating point number
period_years = 8

In [16]:
df, df_clean = utils.download_stonk_prices(
    ticker_list.index, period_years=period_years, date_to=date_to
)
vix, vix_clean = utils.download_stonk_prices(
    ["^VIX"], period_years=period_years, date_to=date_to, fname_prefix="vix"
)
sp500, sp500_clean = utils.download_stonk_prices(
    ["^GSPC"], period_years=period_years, date_to=date_to, fname_prefix="sp500"
)

[*********************100%***********************]  2820 of 2820 completed

16 Failed downloads:
- DELL WI: No data found, symbol may be delisted
- BLL: No data found, symbol may be delisted
- RXN WI: No data found, symbol may be delisted
- O.WI: No data found, symbol may be delisted
- SNX.WI: No data found, symbol may be delisted
- FOE: No data found, symbol may be delisted
- SGMS: No data found, symbol may be delisted
- BIP.PRB: No data found, symbol may be delisted
- T WD: No data found, symbol may be delisted
- BIP.PRA: No data found, symbol may be delisted
- PFE.WI: No data found, symbol may be delisted
- EPAY: No data found, symbol may be delisted
- JOBS: No data found, symbol may be delisted
- MRK.WI: No data found, symbol may be delisted
- MGP: No data found, symbol may be delisted
- POST WI: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Run data pipeline

In [2]:
industries = [
    # 'health_care_equipment_and_services',
    # 'software_and_services',
    # 'retailing',
    # 'telecommunication_services',
    "capital_goods",
    # 'energy',
    # 'pharmaceuticals_biotechnology_and_life_sciences',
    # 'consumer_staples',
    # 'banks',
    # 'diversified_financials',
    # 'metals_and_mining',
    # 'technology_hardware_and_equipment',
    # 'utilities',
    # 'chemicals',
    # 'automobiles_and_components',
    # 'semiconductors_and_semiconductor_equipment',
    # 'media_and_entertainment',
    # 'real_estate',
    # 'consumer_services',
    # 'consumer_durables_and_apparel',
    # 'insurance',
    # 'transportation',
    # 'commercial_and_professional_services',
    # 'paper_and_forest_products',
    # 'containers_and_packaging',
    # 'construction_materials'
]

l_reg = 3
l_roll = 2
dt = 10

output_dir = "data"

stonk_model = predict.XGBStonkModel()
vix = utils.get_stonk_data(fname_prefix="vix", disable_filter=True).iloc[0]

In [3]:
datasets = []
i = 1
total_industries = len(industries)
for industry in industries:
    stonks = utils.get_stonk_data(filter_industries=[industry])
    X, Y = processing.combine_stonk_pairs(stonks)

    print("Industry ({0}/{1}): {2}".format(i, total_industries, industry))

    print("Processing residuals...")
    residuals, betas, _, date_index = utils.measure_time(
        partial(
            processing.get_rolling_residuals,
            X=X,
            Y=Y,
            l_reg=l_reg,
            l_roll=l_roll,
            dt=dt,
        )
    )
    residuals.insert(0, "dates", date_index)
    betas.insert(0, "dates", date_index)

    std_residuals, means, stds = processing.get_standardized_residuals(
        residuals.drop(columns="dates")
    )

    trades_before = len(std_residuals)
    std_residuals = std_residuals[std_residuals.iloc[:, -1].abs() >= 2.5]
    trades_after = len(std_residuals)
    print(
        "{0} trades selected out of {1} by residual values".format(
            trades_after, trades_before
        )
    )
    if trades_after == 0:
        print("No trades left after filtering residuals, skipping this industry...")
        continue
    residuals = residuals.loc[std_residuals.index]
    betas = betas.loc[std_residuals.index]

    print("Processing ADFs...")
    adfs, adfs_raw = utils.measure_time(
        partial(
            processing.get_aggregate_adfs,
            residuals.drop(columns="dates"),
            betas=betas.drop(columns="dates"),
        )
    )

    selected_by_adf = (adfs >= 0.5).values
    adfs = adfs[selected_by_adf]

    trades_before = len(std_residuals)
    std_residuals = std_residuals[selected_by_adf]
    trades_after = len(std_residuals)
    print(
        "{0} trades selected out of {1} by ADF pass rates".format(
            trades_after, trades_before
        )
    )

    if len(std_residuals) == 0:
        print(
            "No trades left after filtering ADF pass rates, skipping this industry..."
        )
        continue

    betas = betas.loc[adfs.index]
    residuals = residuals.loc[adfs.index]
    adfs_raw = adfs_raw.loc[adfs.index]
    means = means.loc[adfs.index]
    stds = stds.loc[adfs.index]

    residuals_max_mean = processing.get_mean_residual_magnitude(
        std_residuals.to_numpy(), dt=21
    )
    print(
        "Mean max residual value for {0} after filtering is {1}".format(
            industry, residuals_max_mean
        )
    )

    print("Preparing data for model...")
    dataset = utils.build_dataset_from_live_data_by_industry(
        std_residuals.to_numpy(),
        adfs.to_numpy().ravel(),
        industry,
        residuals_max_mean,
        vix.loc[stonks.columns[-1]],
    )

    print("Running model...")
    predictions, df_processed = stonk_model.predict(dataset)
    datasets.append((dataset, df_processed))
    predictions = pd.DataFrame(predictions)
    predictions.index = adfs.index

    print("Writing results to CSV...")
    # Very big industry, exceeds Git file size limit
    if industry == "diversified_financials":
        half = len(residuals) // 2
        residuals_fst = residuals.iloc[:half]
        residuals_snd = residuals.iloc[half:]
        residuals_fst.to_csv(
            os.path.join(output_dir, industry + "_one_residuals.csv"),
            header=False,
            index=True,
        )
        residuals_snd.to_csv(
            os.path.join(output_dir, industry + "_two_residuals.csv"),
            header=False,
            index=True,
        )
        del residuals_fst
        del residuals_snd
    else:
        residuals.to_csv(
            os.path.join(output_dir, industry + "_residuals.csv"),
            header=False,
            index=True,
        )
    betas.to_csv(
        os.path.join(output_dir, industry + "_betas.csv"), header=False, index=True
    )
    adfs_raw.to_csv(
        os.path.join(output_dir, industry + "_adfs_raw.csv"), header=False, index=True
    )
    predictions.to_csv(
        os.path.join(output_dir, industry + "_predictions.csv"),
        header=False,
        index=True,
    )
    i += 1

print("*** All done ***")

Industry (1/1): capital_goods
Processing residuals...
Done after: 33s
807 trades selected out of 16836 by residual values
Processing ADFs...
Done after: 80s
166 trades selected out of 807 by ADF pass rates
Mean max residual value for capital_goods after filtering is 3.869999885559082
Preparing data for model...
Running model...
Writing results to CSV...
*** All done ***


# Data collection

In [2]:
stonks = utils.get_stonk_data()
# stonks = stonks.loc[:, :"2022-07-01"]

In [ ]:
pipelines.data_collection_rolling_pipeline(
    stonks,
    l_reg=3,
    l_roll=2,
    dt=10,
    market_cap_min_mm=1000,
    market_cap_max_mm=None,
    last_residual_cutoff=2.5,
    mean_max_residual_dt=21,
    adf_pval_cutoff=0.1,
    adf_pass_rate_filter=0.5,
    arima_forecast_months=3,
    arima_eval_models=5,
    trade_length_months=3,
    trading_interval_weeks=2,
    # first_n_windows=72,
)

Total data windows: 70
Period 2017-04-28 to 2022-07-15
Industries 1/26
Industries 2/26
Industries 3/26
Industries 4/26
Industries 5/26
Industries 6/26
Industries 7/26
Industries 8/26


In [ ]:
dataset = utils.ingest_trade_pipeline_outputs()

vix = utils.get_stonk_data(fname_prefix="vix", disable_filter=True).iloc[0]
sp500 = utils.get_stonk_data(fname_prefix="sp500", disable_filter=True).iloc[0]

sp500_chg = pd.Series((sp500.iloc[63:].values / sp500.iloc[:-63].values) - 1)
sp500_chg.index = sp500.iloc[63:].index

dataset["vix"] = dataset["trade_date"].apply(lambda x: vix.loc[x])
dataset["sp500"] = dataset["trade_date"].apply(lambda x: sp500_chg.loc[x])
dataset.to_csv("data/dataset.csv", header=True, index=False)

# Model development

In [ ]:
import xgboost as xgb
from hyperopt import STATUS_OK, STATUS_FAIL, Trials, fmin, hp, tpe, atpe, rand
import pickle

In [ ]:
def train_production_xgb(
    df: pd.DataFrame, params: Dict[str, Any], noise_level: float = 0
) -> Tuple[xgb.XGBClassifier, sklearn.base.TransformerMixin]:
    X_train, scalers = preprocessing.transform_features(df, noise_level=noise_level)
    y_train = df["label"]

    clf = xgb.XGBClassifier(**params)

    clf.fit(X_train, y_train, eval_set=[(X_train, y_train)])
    clf.save_model(os.path.join("data", "xgb_classifier.json"))

    with open(os.path.join("data", "scalers.json"), "wb") as fp:
        pickle.dump(scalers, fp)

    return clf, scalers

In [ ]:
df = pd.read_csv("data/dataset.csv")
df = df[df.beta > 0]
df = df[df.last_residual.abs() >= 2.5]
df = preprocessing.assign_labels(df)

In [ ]:
drop_dates = 2
selected_dates = np.sort(df["trade_date"].unique())[drop_dates:]
df_prod = df[df.trade_date.isin(selected_dates)].sample(frac=1)
print(len(df_prod))
print(df_prod["label"].value_counts())

In [ ]:
clf_prod, scalers_prod = train_production_xgb(df_prod, params, noise_level=0.005)

In [ ]:
splits = preprocessing.split_data(df, 2, 6, 2, random_state=3439)
print(len(splits["train"]))
print(len(splits["validation"]))
print(splits["train"]["label"].value_counts())
print(splits["validation"]["label"].value_counts())

In [ ]:
noise_level = 0.005

X_train, scalers = preprocessing.transform_features(
    splits["train"], noise_level=noise_level
)
X_valid, _ = preprocessing.transform_features(
    splits["validation"], scalers=scalers, noise_level=0
)

y_train = splits["train"]["label"]
y_valid = splits["validation"]["label"]

In [ ]:
hyperparameter_space = {
    "gamma": hp.uniform("gamma", 0, 5),
    "scale_pos_weight": hp.uniform("scale_pos_weight", 2, 12),
    "max_depth": hp.quniform("max_depth", 3, 10, 1),
    "min_child_weight": hp.quniform("min_child_weight", 1, 8, 1),
    "max_delta_step": hp.quniform("max_delta_step", 1, 4, 1),
    "n_estimators": hp.quniform("n_estimators", 50, 200, 1),
    # "n_estimators": hp.choice("n_estimators", np.array([50, 75, 100, 150, 200])),
    # "subsample": hp.uniform("subsample", 0.5, 1),
    # "colsample_bylevel" : hp.uniform("colsample_bylevel", 0.5, 1),
}

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


def optimization_objective(space):
    clf = xgb.XGBClassifier(
        gamma=space["gamma"],
        scale_pos_weight=space["scale_pos_weight"],
        #
        max_depth=int(space["max_depth"]),
        min_child_weight=int(space["min_child_weight"]),
        max_delta_step=int(space["max_delta_step"]),
        #
        # colsample_bylevel = space['colsample_bylevel'],
        colsample_bylevel=1,
        n_estimators=int(space["n_estimators"]),
        learning_rate=0.1,
        # subsample = space['subsample'],
        subsample=1,
        #
        tree_method="hist",
        enable_categorical=True,
        max_cat_to_onehot=1,
        random_state=np.random.randint(9999999),
    )

    clf.fit(
        X_train,
        y_train,
        verbose=False,
    )

    y_score = clf.predict_proba(X_valid)[:, 1]
    y_preds = y_score > 0.5

    f1 = f1_score(y_valid, y_preds, zero_division=0)
    precision = precision_score(y_valid, y_preds, zero_division=0)
    ap = evaluate.average_precision_from_cutoff(y_valid, y_score, 0.4)
    roc = roc_auc_score(y_valid, y_score)

    pos_preds = int(y_preds.sum())
    pos_labels = int(y_valid.sum())

    ap = ap if pos_preds >= pos_labels else 0

    if f1 == 0 or precision == 0:
        return {
            "loss": 100,
            "precision": precision,
            "f1_score": f1,
            "ap": ap,
            "auc": roc,
            "pos_preds": pos_preds,
            "status": STATUS_FAIL,
        }
    else:
        return {
            "loss": -ap,
            "precision": precision,
            "f1_score": f1,
            "ap": ap,
            "auc": roc,
            "pos_preds": pos_preds,
            "status": STATUS_OK,
        }

In [ ]:
trials = Trials()

best_hyperparams = fmin(
    fn=optimization_objective,
    space=hyperparameter_space,
    algo=tpe.suggest,
    max_evals=1000,
    trials=trials,
)

trial_vals = trials.vals
trial_vals["f1_score"] = list(map(lambda x: x["f1_score"], trials.results))
trial_vals["precision"] = list(map(lambda x: x["precision"], trials.results))
trial_vals["ap"] = list(map(lambda x: x["ap"], trials.results))
trial_vals["auc"] = list(map(lambda x: x["auc"], trials.results))
trial_vals["pos_preds"] = list(map(lambda x: x["pos_preds"], trials.results))

df_trials = pd.DataFrame.from_dict(trial_vals)
df_trials.to_csv("data/data-window-size-2#7.csv", index=False)

In [ ]:
params = {
    # reg def 0
    "gamma": 3.301387,
    # L2 def 1
    # "reg_lambda" : 1,
    # "reg_alpha" : 0,
    # Class imbalance def 1
    "scale_pos_weight": 5.568589,
    # Integers:
    "max_depth": 6,
    # Reg def 1
    "min_child_weight": 7,
    # Class imbalance def 0
    "max_delta_step": 3,
    # Choice:
    "colsample_bylevel": 1,
    "n_estimators": 63,
    "learning_rate": 0.1,
    "subsample": 1,
    # Fixed:
    "tree_method": "hist",
    "enable_categorical": True,
    "max_cat_to_onehot": 1,
    "eval_metric": ["logloss"],
    "random_state": np.random.randint(999929),
}

clf = xgb.XGBClassifier(**params)

clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid), (X_train, y_train)])

In [ ]:
print("**Validation**")
y_score = clf.predict_proba(X_valid)[:, 1]
thres = 0.7
y_preds = y_score > thres

evaluate.performance_summary(y_score, y_preds, y_valid, auc_cutoff=0.4)

df_results_valid = evaluate.returns_on_predictions(splits["validation"], y_preds)

evaluate.performance_on_slice(
    splits["validation"], y_score, y_preds, "subindustry", False
)

In [ ]:
pd.set_option("display.max_rows", 100)

In [ ]:
# df_results_valid[df_results_valid.result == 'TP'].iloc[:100]

In [ ]:
# df_results_valid[df_results_valid.subindustry == 'consumer_services'].iloc[0:100]

In [ ]:
print(clf.feature_names_in_)

In [ ]:
print(clf.feature_importances_)

In [ ]:
df_trials = pd.read_csv("data/last-residual-cutoff-check-2.5#13.csv")
df_trials.sort_values("ap", ascending=False).head(50)

In [ ]:
df_trials = pd.read_csv("data/data-window-size-2#7.csv")
df_trials.sort_values("ap", ascending=False).head(50)

In [ ]:
df_trials = pd.read_csv("data/data-window-size-2#6.csv")
df_trials.sort_values("ap", ascending=False).head(50)